<a href="https://colab.research.google.com/github/anupmalh/demo/blob/master/LLMs/langchain-example/automatic_grader-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example Code: Automatic Grader

Code authored by: Shawhin Talebi <br>
Blog link: https://medium.com/towards-data-science/prompt-engineering-how-to-trick-ai-into-solving-your-problems-7ce1ed3b553f

### import libraries

In [1]:
!pip install openai==0.27.8
!pip install tqdm==4.66.1
!pip install langchain[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.6/120.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Suc

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

In [3]:
!pip install sk

  Preparing metadata (setup.py) ... done
  Created wheel for sk: filename=sk-0.0.1-py3-none-any.whl size=1869 sha256=94642d28b79070ff3253f7d5d5415a2183b9fd5278e2dc45124f43f75df0d984
  Stored in directory: /root/.cache/pip/wheels/1f/c8/ef/12b6b826e0dbc9e760d19c65b967a6cef40c56db145d4717d6
Successfully built sk


In [9]:
my_sk = ""

In [ ]:
# import sk
from sk import my_sk

ImportError: cannot import name 'my_sk' from 'sk' (/usr/local/lib/python3.10/dist-packages/sk/__init__.py)

### define chain

In [10]:
# define chatbot
chat_model = ChatOpenAI(openai_api_key=my_sk, temperature=0)

In [11]:
# define prompt template
prompt_template_text = """You are a high school history teacher grading homework assignments. \
Based on the homework question indicated by “**Q:**” and the correct answer indicated by “**A:**”, your task is to determine whether the student's answer is correct. \
Grading is binary; therefore, student answers can be correct or wrong. \
Simple misspellings are okay.

**Q:** {question}
**A:** {correct_answer}

**Student's Answer:** {student_answer}
"""

prompt = PromptTemplate(input_variables=["question", "correct_answer", "student_answer"], template = prompt_template_text)

In [12]:
# define chain
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
)

### use chain

In [13]:
# define inputs
question = "Who was the 35th president of the United States of America?"
correct_answer = "John F. Kennedy"
student_answer =  "JFK"

# run chain
chain.run({'question':question, 'correct_answer':correct_answer, 'student_answer':student_answer})

'Correct'

In [14]:
# run chain in for loop
student_answer_list = ["John F. Kennedy", "JFK", "FDR", "John F. Kenedy", "John Kennedy", "Jack Kennedy", "Jacqueline Kennedy", "Robert F. Kenedy"]

for student_answer in student_answer_list:
    print(student_answer + " - " + str(chain.run({'question':question, 'correct_answer':correct_answer, 'student_answer':student_answer})))
    print('\n')

John F. Kennedy - The student's answer is correct.


JFK - **Grade: Correct**


FDR - The student's answer is wrong. The 35th president of the United States of America was John F. Kennedy, not FDR.


John F. Kenedy - Correct


John Kennedy - The student's answer is correct.


Jack Kennedy - **Grade: Correct** - The student's answer "Jack Kennedy" is correct, as it is a common nickname for John F. Kennedy, who was indeed the 35th president of the United States.


Jacqueline Kennedy - **Student's Answer:** Jacqueline Kennedy

**Grade:** Wrong

**Feedback:** The correct answer is John F. Kennedy, not Jacqueline Kennedy.


Robert F. Kenedy - The student's answer is incorrect.




### add output parser

In [15]:
# define output parser
class GradeOutputParser(BaseOutputParser):
    """Determine whether grade was correct or wrong"""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return "wrong" not in text.lower()

In [16]:
# update chain
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    output_parser=GradeOutputParser()
)

In [17]:
# grade student answers
for student_answer in student_answer_list:
    print(student_answer + " - " + str(chain.run({'question':question, 'correct_answer':correct_answer, 'student_answer':student_answer})))

John F. Kennedy - True
JFK - True
FDR - False
John F. Kenedy - True
John Kennedy - True
Jack Kennedy - True
Jacqueline Kennedy - False
Robert F. Kenedy - True
